In [10]:
from random import random, seed
from itertools import product
import numpy as np
from icecream import ic

In [11]:
UNIVERSE_SIZE=20
NUM_SETS=10
DENSITY=0.1

rng= np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000*DENSITY)]))


In [12]:
SETS=np.random.random((NUM_SETS, UNIVERSE_SIZE)) <DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s]= True

COSTS=np.pow(SETS.sum(axis=1), 1.1)
COSTS

array([4.59479342, 2.14354693, 3.34836952, 1.        , 3.34836952,
       2.14354693, 7.17738719, 2.14354693, 1.        , 4.59479342])

In [13]:
def valid(solution):
    return np.all(np.logical_or.reduce(SETS[solution]))

def cost(solution):
    return COSTS[solution].sum()

In [14]:
#dumb solution of "all" sets
solution=np.full(NUM_SETS, True)
ic(valid(solution), cost(solution))
None


ic| valid(solution): np.True_
    cost(solution): np.float64(31.49435385250536)


In [15]:
#a random solution with random 50% of the sets
solution=rng.random(NUM_SETS)<0.5
valid(solution), cost(solution)
None

# Simple RHMC

In [16]:
def single_mutation(solution: np.ndarray)-> np.ndarray:
    new_sol=solution.copy()
    i=rng.integers(0, NUM_SETS)
    new_sol[i]=not new_sol[i]
    return new_sol

def multiple_mutation(solution: np.ndarray, strength: float=0.3)-> np.ndarray:
    new_sol=solution.copy()
    mask=rng.random(NUM_SETS)<strength
    if not np.any(mask):
        mask[np.random.randint(NUM_SETS)]=True
    new_sol=np.logical_xor(solution, mask)
    return new_sol

In [17]:
def fitness(solution: np.ndarray):
    return (valid(solution), -cost(solution))

In [18]:

solution=rng.random(NUM_SETS)<0.3
solution_fitness=fitness(solution)
ic(fitness(solution))


tweak=multiple_mutation

strength=0.5
for steps in range(100):
    if steps%1_000:
        strength/=2
    new_solution=multiple_mutation(solution, strength)
    if fitness(new_solution)>solution_fitness:
        solution=new_solution
        solution_fitness=fitness(solution)

ic(fitness(solution))

ic| fitness(solution): (np.False_, np.float64(-8.635463372246885))
ic| fitness(solution): (np.False_, np.float64(-0.0))

(np.False_, np.float64(-0.0))